<a href="https://colab.research.google.com/github/husnain-ali21/transformers_for_nlp/blob/main/Video_Captioning_with_wave2vec_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install transformers
!pip -q install youtube_dl

     |████████████████████████████████| 2.1MB 6.8MB/s 
     |████████████████████████████████| 3.3MB 39.1MB/s 
     |████████████████████████████████| 870kB 43.7MB/s 
     |████████████████████████████████| 1.9MB 9.2MB/s 


In [2]:
from transformers import Wav2Vec2Tokenizer ,Wav2Vec2ForCTC
from IPython.display import Audio
import moviepy.editor  as mp # Python library for video editing: cutting, concatenations, video processin
import torch 
import librosa # python package for music and audio analysis.
import os

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2686976/45929032 bytes (5.9%)6029312/45929032 bytes (13.1%)9723904/45929032 bytes (21.2%)13082624/45929032 bytes (28.5%)16621568/45929032 bytes (36.2%)20455424/45929032 bytes (44.5%)24412160/45929032 bytes (53.2%)27680768/45929032 bytes (60.3%)31006720/45929032 bytes (67.5%)34488320/45929032 bytes (75.1%)38068224/45929032 bytes (82.9%)41574400/45929032 bytes (90.5%)

In [9]:
video = "https://www.youtube.com/watch?v=rXGqKJoQ4qM&ab_channel=TheDenverPost"  #link of video

os.system(f'youtube-dl {video} --recode-video mp4')
os.system('mv *.mp4 video.mp4')

256

In [5]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
#First we'll extract the audio in mp3 format from the clip, as the Wav2Vec models expects audio input.
# We do this in subclips of 10 second length to save some memory later on. 

clip = mp.VideoFileClip("/content/Machine learning explained in 1 minute and 20 seconds using a watermelon-rXGqKJoQ4qM.mkv")
end = clip.duration 

clip_paths = []

#extract audio from video
for i in range(1 ,int(end) ,10):
    sub_end = min(i+10 , end)
    sub_clip = clip.subclip(i , sub_end)
    path = f"audio_{i}_.mp3"
    sub_clip.audio.write_audiofile(path)   
    clip_paths.append(path)


[MoviePy] Writing audio in audio_1_.mp3


100%|██████████| 221/221 [00:00<00:00, 602.00it/s]

[MoviePy] Done.


[MoviePy] Writing audio in audio_11_.mp3


100%|██████████| 221/221 [00:00<00:00, 513.15it/s]

...

[MoviePy] Writing audio in audio_61_.mp3


100%|██████████| 221/221 [00:00<00:00, 550.55it/s]


[MoviePy] Done.
[MoviePy] Writing audio in audio_71_.mp3


100%|██████████| 218/218 [00:00<00:00, 567.72it/s]

[MoviePy] Done.


In [23]:
Audio(clip_paths[5])

In [16]:
#now we turn audio into text with wav2vec
transcription = ""
for path in clip_paths:
    input_audio , _ = librosa.load(path) # loading the audio
    audio_tokens = tokenizer(input_audio , return_tensors = "pt"  , padding="longest").input_values

    with torch.no_grad():
        logits = model(audio_tokens).logits
        preds = torch.argmax(logits , dim = -1)
    caption = tokenizer.batch_decode(preds)[0]
    transcription += caption + " "

    



In [ ]:
print(transcription)